# Load FHIBE Dataset

![img](../images/fhibe.png)

This notebook loads the Sony AI's "Fair Human-Centric Image Benchmark" dataset as a 3LC Table, including keypoints, segmentation, bounding boxes, as well as rich subject metadata.

<!-- Tags: ["keypoints", "instance-segmentation", "object-detection"] -->

To download the dataset, you need to register at [fairnessbenchmark.ai.sony](https://fairnessbenchmark.ai.sony/). To read the original research paper, check see [here](https://www.nature.com/articles/s41586-025-09716-2).

Several versions of the dataset exist, for this tutorial we will use version contained in `fhibe.20250716.u.gT5_rFTA_downsampled_public.tar.gz`, but the ingestion script should work for any version of the dataset. 

We have tried to include as much as possible of the metadata contained in the dataset, omitting only a few attributes in the name of simplicity, e.g. the `<attr>_QA_annotator_id` fields have been left out.

The data can be categorized as follows:
- Main image
- Geometric annotations (instance segmentations, keypoints, facial bounding box)
- Image-level metadata (shutter speed, camera manufacturer, weather, etc.)
- Subject-level metadata (ancestry, hair color, age, etc.)

This script reads all this data from per-subject JSON files and converts it to a format suitable for a 3LC Table. Several of the columns are stored as "categorical strings" (e.g. hair color "Blond", "Gray", "White", ...), these values are converted to integers, with their corresponding string values stored in the schema. This makes it easier to filter and work with these values in the 3LC Dashboard.

## Install dependencies

In [ ]:
%pip install -q 3lc

## Imports

In [ ]:
import json
from pathlib import Path

import numpy as np
import tlc
from tqdm import tqdm

## Project setup

In [ ]:
PROJECT_NAME = "3LC Tutorials - FHIBE"
DATASET_NAME = "FHIBE"
TABLE_NAME = "initial"
MAX_SAMPLES = None
DOWNLOAD_PATH = "D:/Data"

## Prepare data

In [ ]:
FHIBE_ROOT = Path(DOWNLOAD_PATH) / "fhibe"
DATA_ROOT = FHIBE_ROOT / "data/raw/fhibe_downsampled"

if not FHIBE_ROOT.exists():
    raise FileNotFoundError(f"FHIBE_ROOT does not exist: {FHIBE_ROOT}")

if not DATA_ROOT.exists():
    raise FileNotFoundError(f"DATA_ROOT does not exist: {DATA_ROOT}")

annotation_paths = list(DATA_ROOT.glob("**/main_annos_*.json"))
print(f"Found {len(annotation_paths)} annotation files")

## Prepare value mappings

### Image-level mappings

In [ ]:
camera_position_value_map = {"Typical": 0, "Atypical High": 1, "Atypical Low": 2}
camera_distance_value_map = {"CD I": 0, "CD II": 1, "CD III": 2, "CD IV": 3, "CD V": 4}
lighting_value_map = {
    "Lighting from above the head/face": 0,
    "Lighting from below the head/face": 1,
    "Lighting from in front of the head/face": 2,
    "Lighting from behind the head/face": 3,
    "Lighting from the left of the head/face": 4,
    "Lighting from the right of the head/face": 5,
}
weather_value_map = {"Fog": 0, "Haze": 1, "Snow/hail": 2, "Rain": 3, "Humid": 4, "Cloud": 5, "Clear": 6}
user_hour_captured_value_map = {"0000-0559": 0, "0600-1159": 1, "1200-1759": 2, "1800-2359": 3}
scene_value_map = {
    "Outdoor Water, ice, snow": 0,
    "Outdoor Mountains, hills, desert, sky": 1,
    "Outdoor Forest, field, jungle": 2,
    "Outdoor Man-made elements": 3,
    "Outdoor Transportation": 4,
    "Outdoor Cultural or historical building/place": 5,
    "Outdoor Sports fields, parks, leisure spaces": 6,
    "Outdoor Industrial and construction": 7,
    "Outdoor Houses, cabins, gardens, and farms": 8,
    "Outdoor Commercial buildings, shops, markets, cities, and towns": 9,
    "Indoor Shopping and dining": 10,
    "Indoor Workplace": 11,
    "Indoor Home or hotel": 12,
    "Indoor Transportation": 13,
    "Indoor Sports and leisure": 14,
    "Indoor Cultural": 15,
}

### Subject-level mappings

In [ ]:
segments_value_map = {
    "Face skin": 0.0,
    "Upper body skin": 1.0,
    "Right eye": 2.0,
    "Nose": 3.0,
    "Upper lip": 4.0,
    "Lower lip": 5.0,
    "Inner mouth": 6.0,
    "Left shoe": 7.0,
    "Right shoe": 8.0,
    "Left arm skin": 9.0,
    "Upper body clothes": 10.0,
    "Lower body clothes": 11.0,
    "Sock or legwarmer": 12.0,
    "Jewelry or timepiece": 13.0,
    "Right arm skin": 14.0,
    "Left leg skin": 15.0,
    "Right leg skin": 16.0,
    "Head hair": 17.0,
    "Left eyebrow": 18.0,
    "Right eyebrow": 19.0,
    "Left eye": 20.0,
    "Bag": 21.0,
    "Eyewear": 22.0,
    "Full body clothes": 23.0,
    "Headwear": 24.0,
    "Mask": 25.0,
    "Neckwear": 26.0,
    "Glove": 27.0,
}

pronoun_value_map = {
    "She/her/hers": 0,
    "He/him/his": 1,
    "They/them/their": 2,
    "Ze/zir/zirs": 3,
    "None of the above": 4,
    "Prefer not to say": 5,
}

head_pose_value_map = {
    "Typical": 0,
    "Atypical": 1,
}

facial_marks_value_map = {
    "None": 0,
    "Tattoos": 1,
    "Birthmarks": 2,
    "Scars": 3,
    "Burns": 4,
    "Growths": 5,
    "Make-up": 6,
    "Face paint": 7,
    "Acne": 8,
    "Not listed": 9,
    "Free-text": 10,
}

ancestry_value_map = {
    "Africa": 0,
    "Eastern Africa": 1,
    "Northern Africa": 2,
    "Middle Africa": 3,
    "Southern Africa": 4,
    "Western Africa": 5,
    "Americas": 6,
    "Caribbean": 7,
    "Central America": 8,
    "South America": 9,
    "Northern America": 10,
    "Asia": 11,
    "Central Asia": 12,
    "Eastern Asia": 13,
    "South-eastern Asia": 14,
    "Southern Asia": 15,
    "Western Asia": 16,
    "Europe": 17,
    "Eastern Europe": 18,
    "Northern Europe": 19,
    "Southern Europe": 20,
    "Western Europe": 21,
    "Oceania": 22,
    "Australia and New Zealand": 23,
    "Polynesia": 24,
}

skin_color_value_map = {
    "[102, 78, 65]": 0,
    "[136, 105, 81]": 1,
    "[164, 131, 103]": 2,
    "[175, 148, 120]": 3,
    "[189, 163, 137]": 4,
    "[198, 180, 157]": 5,
}

haircolor_value_map = {
    "None": 0,
    "Very light blond": 1,
    "Light blond": 2,
    "Blond": 3,
    "Dark blond": 4,
    "Light brown to medium brown": 5,
    "Dark brown/black": 6,
    "Red": 7,
    "Gray": 8,
    "Red blond": 9,
    "White": 10,
    "Not listed": 11,
    "Free-text": 12,
}

hairstyle_value_map = {
    "None": 0,
    "Buzz cut": 1,
    "Up (Short)": 10,
    "Half-up (Short)": 11,
    "Down (Short)": 12,
    "Not listed(Short)": 13,
    "Up (Medium)": 14,
    "Half-up (Medium)": 15,
    "Down (Medium)": 2,
    "Not listed(Medium)": 3,
    "Up (Long)": 4,
    "Half-up (Long)": 5,
    "Down (Long)": 6,
    "Not listed(Long)": 7,
    "Not listed": 8,
    "Free-text": 9,
}

facial_hairstyle_value_map = {
    "None": 0,
    "Beard": 1,
    "Mustache": 2,
    "Goatee": 3,
}

hair_type_value_map = {
    "None": 0,
    "Straight": 1,
    "Wavy": 2,
    "Curly": 3,
    "Kinky-coily": 4,
    "Not listed": 5,
    "Free-text": 6,
}

action_body_pose_value_map = {
    "Standing": 0,
    "Sitting": 1,
    "Walking": 2,
    "Bending/bowing": 3,
    "Lying down/sleeping": 4,
    "Performing martial/fighting arts": 5,
    "Dancing": 6,
    "Running/jogging": 7,
    "Crouching/kneeling": 8,
    "Getting up": 9,
    "Jumping/leaping": 10,
    "Falling down": 11,
    "Crawling": 12,
    "Swimming": 13,
    "Not listed": 14,
    "Free-text": 15,
}

action_subject_object_interaction_value_map = {
    "None": 0,
    "Riding": 1,
    "Driving": 2,
    "Watching": 3,
    "Smoking": 4,
    "Eating": 5,
    "Drinking": 6,
    "Opening or closing": 7,
    "Lifting/picking up or putting down": 8,
    "Writing/drawing or painting": 9,
    "Catching or throwing": 10,
    "Pushing, pulling or extracting": 11,
    "Putting on or taking off clothing": 12,
    "Entering or exiting": 13,
    "Climbing": 14,
    "Pointing at": 15,
    "Shooting at": 16,
    "Digging/shoveling": 17,
    "Playing with pets/animals": 18,
    "Playing musical instrument": 19,
    "Playing": 20,
    "Using an electronic device": 21,
    "Cutting or chopping": 22,
    "Cooking": 23,
    "Fishing": 24,
    "Rowing": 25,
    "Sailing": 26,
    "Brushing teeth": 27,
    "Hitting": 28,
    "Kicking": 29,
    "Turning": 30,
    "Not listed": 31,
    "Free-text": 32,
}

eye_color_value_map = {
    "None": 0,
    "Blue": 1,
    "Gray": 2,
    "Green": 3,
    "Hazel": 4,
    "Brown": 5,
    "Red and violet": 6,
    "Not listed": 7,
    "Free-text": 8,
}

## Define data processing steps

In [ ]:
def clean_str(s):
    if ". " in s:
        # Many FHIBE strings are numbered, e.g. "1. Right eye inner". For readability, remove the numbering
        s = s.split(". ")[1]
    # MapElements in 3LC do not support ":" in the name
    return s.replace(":", "")

In [ ]:
NUM_KEYPOINTS = 33
SKELETON = [
    11,
    12,
    11,
    13,
    13,
    15,
    12,
    14,
    14,
    16,
    12,
    24,
    11,
    23,
    23,
    24,
    24,
    26,
    26,
    28,
    23,
    25,
    25,
    27,
    27,
    29,
    29,
    31,
    28,
    30,
    30,
    32,
]

KEYPOINTS = [
    "Nose",
    "Right eye inner",
    "Right eye",
    "Right eye outer",
    "Left eye inner",
    "Left eye",
    "Left eye outer",
    "Right ear",
    "Left ear",
    "Mouth right",
    "Mouth left",
    "Right shoulder",
    "Left shoulder",
    "Right elbow",
    "Left elbow",
    "Right wrist",
    "Left wrist",
    "Right pinky knuckle",
    "Left pinky knuckle",
    "Right index knuckle",
    "Left index knuckle",
    "Right thumb knuckle",
    "Left thumb knuckle",
    "Right hip",
    "Left hip",
    "Right knee",
    "Left knee",
    "Right ankle",
    "Left ankle",
    "Right heel",
    "Left heel",
    "Right foot index",
    "Left foot index",
]


def process_keypoints(keypoints, image_width, image_height):
    keypoints = {clean_str(kpt_name): v for kpt_name, v in keypoints.items()}
    kpts_arr = np.zeros((NUM_KEYPOINTS, 3), dtype=np.float32)
    for i, kpt_name in enumerate(KEYPOINTS):
        if kpt_name not in keypoints:
            continue
        x, y, viz = keypoints[kpt_name]
        viz = 2 if viz else 0
        kpts_arr[i, :] = [x, y, viz]

    instances = tlc.Keypoints2DInstances.create_empty(
        image_width=image_width,
        image_height=image_height,
        include_keypoint_visibilities=True,
        include_instance_bbs=False,
    )

    instances.add_instance(
        keypoints=kpts_arr,
        label=0,
    )

    return instances.to_row()


def process_segments(segments, image_width, image_height):
    polygons = []
    labels = []

    for segment in segments:
        class_name = clean_str(segment["class_name"])
        polygon = segment["polygon"]
        poly_2_tuples = [[p["x"], p["y"]] for p in polygon]
        flattened_poly = [item for sublist in poly_2_tuples for item in sublist]
        polygons.append(flattened_poly)
        labels.append(segments_value_map[class_name])

    segs = tlc.SegmentationPolygonsDict(
        image_width=image_width,
        image_height=image_height,
        polygons=polygons,
        instance_properties={"label": labels},
    )
    return segs


def process_bboxes(face_bbox, image_width, image_height):
    bboxes = {
        tlc.IMAGE_WIDTH: image_width,
        tlc.IMAGE_HEIGHT: image_height,
        tlc.BOUNDING_BOX_LIST: [
            {
                tlc.X0: face_bbox[0],
                tlc.Y0: face_bbox[1],
                tlc.X1: face_bbox[2],
                tlc.Y1: face_bbox[3],
                tlc.LABEL: 0,
            },
        ],
    }

    return bboxes

In [ ]:
def process_image_annotation(image_annotation):
    aperture_value = image_annotation["aperture_value"]  # float
    camera_distance = camera_distance_value_map[clean_str(image_annotation["camera_distance"])]  # str with value map
    camera_position = camera_position_value_map[clean_str(image_annotation["camera_position"])]  # str with value map
    focal_length = image_annotation["focal_length"]  # float
    iso_speed_ratings = image_annotation["iso_speed_ratings"]  # int
    lighting = [lighting_value_map[clean_str(li)] for li in image_annotation["lighting"]]  # list of str
    location_country = image_annotation["location_country"]  # str
    location_region = image_annotation["location_region"]  # str
    manufacturer = image_annotation["manufacturer"]  # str
    model = image_annotation["model"]  # str
    scene = scene_value_map[clean_str(image_annotation["scene"])]  # str
    shutter_speed_value = image_annotation["shutter_speed_value"]  # float
    user_date_captured = image_annotation["user_date_captured"]  # str
    user_hour_captured = user_hour_captured_value_map[clean_str(image_annotation["user_hour_captured"])]  # str
    weather = [weather_value_map[clean_str(w)] for w in image_annotation["weather"]]  # list of str

    image_annotation_dict = {
        "aperture_value": aperture_value,
        "camera_distance": camera_distance,
        "camera_position": camera_position,
        "focal_length": focal_length,
        "iso_speed_ratings": iso_speed_ratings,
        "lighting": lighting,
        "location_country": location_country,
        "location_region": location_region,
        "manufacturer": manufacturer,
        "model": model,
        "scene": scene,
        "shutter_speed_value": shutter_speed_value,
        "user_date_captured": user_date_captured,
        "user_hour_captured": user_hour_captured,
        "weather": weather,
    }

    return image_annotation_dict

In [ ]:
def process_subject_annotation(subject_annotation, image_height, image_width):
    keypoints = process_keypoints(subject_annotation["keypoints"], image_height, image_width)
    segments = process_segments(subject_annotation["segments"], image_height, image_width)
    bboxes = process_bboxes(subject_annotation["face_bbox"], image_height, image_width)
    subject_id = subject_annotation["subject_id"]  # str
    age = subject_annotation["age"]  # int
    nationality = subject_annotation["nationality"]  # list of str
    ancestry = [ancestry_value_map[clean_str(a)] for a in subject_annotation["ancestry"]]  # list of str
    pronoun = [pronoun_value_map[clean_str(p)] for p in subject_annotation["pronoun"]]  # list of str
    natural_skin_color = skin_color_value_map[clean_str(subject_annotation["natural_skin_color"])]  # categorical str
    apparent_skin_color = skin_color_value_map[clean_str(subject_annotation["apparent_skin_color"])]  # categorical str
    hairstyle = hairstyle_value_map[clean_str(subject_annotation["hairstyle"])]  # categorical str
    natural_hair_type = hair_type_value_map[clean_str(subject_annotation["natural_hair_type"])]  # categorical str
    apparent_hair_type = hair_type_value_map[clean_str(subject_annotation["apparent_hair_type"])]  # categorical str
    natural_hair_color = [
        haircolor_value_map[clean_str(h)] for h in subject_annotation["natural_hair_color"]
    ]  # list of categorical str
    apparent_hair_color = [
        haircolor_value_map[clean_str(h)] for h in subject_annotation["apparent_hair_color"]
    ]  # list of categorical str
    facial_hairstyle = [
        facial_hairstyle_value_map[clean_str(f)] for f in subject_annotation["facial_hairstyle"]
    ]  # list of categorical str
    natural_facial_hair_color = [
        haircolor_value_map[clean_str(h)] for h in subject_annotation["natural_facial_haircolor"]
    ]  # list of categorical str
    apparent_facial_hair_color = [
        haircolor_value_map[clean_str(h)] for h in subject_annotation["apparent_facial_haircolor"]
    ]  # list of categorical str
    natural_left_eye_color = [
        eye_color_value_map[clean_str(h)] for h in subject_annotation["natural_left_eye_color"]
    ]  # list of categorical str
    apparent_left_eye_color = [
        eye_color_value_map[clean_str(h)] for h in subject_annotation["apparent_left_eye_color"]
    ]  # list of categorical str
    natural_right_eye_color = [
        eye_color_value_map[clean_str(h)] for h in subject_annotation["natural_right_eye_color"]
    ]  # list of categorical str
    apparent_right_eye_color = [
        eye_color_value_map[clean_str(h)] for h in subject_annotation["apparent_right_eye_color"]
    ]  # list of categorical str
    facial_marks = [
        facial_marks_value_map[clean_str(f)] for f in subject_annotation["facial_marks"]
    ]  # list of categorical str
    action_body_pose = action_body_pose_value_map[clean_str(subject_annotation["action_body_pose"])]  # categorical str
    action_subject_object_interaction = [
        action_subject_object_interaction_value_map[clean_str(a)]
        for a in subject_annotation["action_subject_object_interaction"]
    ]  # list of categorical str
    head_pose = head_pose_value_map[clean_str(subject_annotation["head_pose"])]  # categorical str

    subject_annotation_dict = {
        "bbs": bboxes,
        "segments": segments,
        "keypoints": keypoints,
        "subject_id": subject_id,
        "age": age,
        "nationality": nationality,
        "ancestry": ancestry,
        "pronoun": pronoun,
        "natural_skin_color": natural_skin_color,
        "apparent_skin_color": apparent_skin_color,
        "hairstyle": hairstyle,
        "natural_hair_type": natural_hair_type,
        "apparent_hair_type": apparent_hair_type,
        "natural_hair_color": natural_hair_color,
        "apparent_hair_color": apparent_hair_color,
        "facial_hairstyle": facial_hairstyle,
        "natural_facial_hair_color": natural_facial_hair_color,
        "apparent_facial_hair_color": apparent_facial_hair_color,
        "natural_left_eye_color": natural_left_eye_color,
        "apparent_left_eye_color": apparent_left_eye_color,
        "natural_right_eye_color": natural_right_eye_color,
        "apparent_right_eye_color": apparent_right_eye_color,
        "facial_marks": facial_marks,
        "action_body_pose": action_body_pose,
        "action_subject_object_interaction": action_subject_object_interaction,
        "head_pose": head_pose,
    }

    return subject_annotation_dict

## Load data

This is the main loop where we iterate over annotation files, extract and process annotations, and store the processed data in a list of rows.

In [ ]:
rows = []

total = len(annotation_paths) if MAX_SAMPLES is None else min(len(annotation_paths), MAX_SAMPLES)

for annotation_path in tqdm(annotation_paths, total=total, desc="Processing annotations"):
    with open(annotation_path) as f:
        annotations = json.load(f)

    image_path = annotation_path.with_name(f"main_{annotations['image']['file_name']}")
    image_annotation = annotations["image_annotation"]
    subject_annotations = annotations["subject_annotation"]
    image_annotation_dict = process_image_annotation(image_annotation)

    for subject_annotation in subject_annotations:
        subject_annotation_dict = process_subject_annotation(
            subject_annotation, image_annotation["image_height"], image_annotation["image_width"]
        )

        rows.append(
            {
                "image": tlc.Url(image_path).to_relative().to_str(),
                **image_annotation_dict,
                **subject_annotation_dict,
            }
        )

        if MAX_SAMPLES is not None and len(rows) >= MAX_SAMPLES:
            break
    else:
        continue

    break  # Max samples reached

## Write 3LC Table

In [ ]:
override_schemas = {
    # Image annotations
    "aperture_value": tlc.Float32Schema(default_visible=False, writable=False),
    "camera_distance": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in camera_distance_value_map.items()}, default_visible=False, writable=False
    ),
    "camera_position": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in camera_position_value_map.items()}, default_visible=False, writable=False
    ),
    "focal_length": tlc.Float32Schema(default_visible=False, writable=False),
    "iso_speed_ratings": tlc.Int32Schema(default_visible=False, writable=False),
    "lighting": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in lighting_value_map.items()}, default_visible=False, writable=False
    ),
    "location_country": tlc.StringSchema(default_visible=False, writable=False),
    "location_region": tlc.StringSchema(default_visible=False, writable=False),
    "manufacturer": tlc.StringSchema(default_visible=False, writable=False),
    "model": tlc.StringSchema(default_visible=False, writable=False),
    "scene": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in scene_value_map.items()}, default_visible=False, writable=False
    ),
    "shutter_speed_value": tlc.Float32Schema(default_visible=False, writable=False),
    "user_date_captured": tlc.StringSchema(default_visible=False, writable=False),
    "user_hour_captured": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in user_hour_captured_value_map.items()}, default_visible=False, writable=False
    ),
    "weather": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in weather_value_map.items()}, default_visible=False, writable=False
    ),
    # Subject annotations
    "subject_id": tlc.StringSchema(default_visible=False, writable=False),
    "age": tlc.Int32Schema(default_visible=False, writable=False),
    "nationality": tlc.StringListSchema(default_visible=False, writable=False),
    "ancestry": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in ancestry_value_map.items()}, default_visible=False, writable=False
    ),
    "pronoun": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in pronoun_value_map.items()}, default_visible=False, writable=False
    ),
    "natural_skin_color": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in skin_color_value_map.items()}, default_visible=False, writable=False
    ),
    "apparent_skin_color": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in skin_color_value_map.items()}, default_visible=False, writable=False
    ),
    "hairstyle": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in hairstyle_value_map.items()}, default_visible=False, writable=False
    ),
    "natural_hair_type": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in hair_type_value_map.items()}, default_visible=False, writable=False
    ),
    "apparent_hair_type": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in hair_type_value_map.items()}, default_visible=False, writable=False
    ),
    "natural_hair_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in haircolor_value_map.items()}, default_visible=False, writable=False
    ),
    "apparent_hair_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in haircolor_value_map.items()}, default_visible=False, writable=False
    ),
    "facial_hairstyle": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in facial_hairstyle_value_map.items()}, default_visible=False, writable=False
    ),
    "natural_facial_hair_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in haircolor_value_map.items()}, default_visible=False, writable=False
    ),
    "apparent_facial_hair_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in haircolor_value_map.items()}, default_visible=False, writable=False
    ),
    "natural_left_eye_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in eye_color_value_map.items()}, default_visible=False, writable=False
    ),
    "apparent_left_eye_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in eye_color_value_map.items()}, default_visible=False, writable=False
    ),
    "natural_right_eye_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in eye_color_value_map.items()}, default_visible=False, writable=False
    ),
    "apparent_right_eye_color": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in eye_color_value_map.items()}, default_visible=False, writable=False
    ),
    "facial_marks": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in facial_marks_value_map.items()}, default_visible=False, writable=False
    ),
    "action_body_pose": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in action_body_pose_value_map.items()}, default_visible=False, writable=False
    ),
    "action_subject_object_interaction": tlc.CategoricalLabelListSchema(
        classes={v: k for k, v in action_subject_object_interaction_value_map.items()},
        default_visible=False,
        writable=False,
    ),
    "head_pose": tlc.CategoricalLabelSchema(
        classes={v: k for k, v in head_pose_value_map.items()}, default_visible=False, writable=False
    ),
}

In [ ]:
table_writer = tlc.TableWriter(
    table_name=TABLE_NAME,
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    column_schemas={
        "image": tlc.ImageUrlSchema(),
        "keypoints": tlc.Keypoints2DSchema(
            classes=["person"],
            num_keypoints=NUM_KEYPOINTS,
            lines=SKELETON,
            point_attributes=KEYPOINTS,
            include_per_point_visibility=True,
        ),
        "bbs": tlc.BoundingBoxListSchema(
            label_value_map={0: tlc.MapElement("face")},
            include_segmentation=False,
            x1_number_role=tlc.NUMBER_ROLE_BB_SIZE_X,
            y1_number_role=tlc.NUMBER_ROLE_BB_SIZE_Y,
        ),
        "segments": tlc.SegmentationSchema(
            label_value_map={v: tlc.MapElement(k) for k, v in segments_value_map.items()},
        ),
        **override_schemas,
    },
)

for row in tqdm(rows, total=len(rows), desc="Writing rows"):
    table_writer.add_row(row)

table = table_writer.finalize()